<a href="https://colab.research.google.com/github/zwubbena/sas-cartesian-product/blob/main/cartesian_product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# Cartesian Product
# =============================================================================

# =============================================================================
# SECTION 1: Setup and Library Imports
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Create an output directory
OUTPUT_DIR = "/content/output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =============================================================================
# SECTION 2: Create 2019 and 2020 Determination Level Tables
# =============================================================================

# Create 2019 DL table (equivalent to a_table in SAS)
a_table = pd.DataFrame({
    'DL': ['DL0', 'DL1', 'DL2', 'DL3', 'DL4'],
    'DL2019': [0, 1, 2, 3, 4]
})

# Create 2020 DL table (equivalent to b_table in SAS)
b_table = pd.DataFrame({
    'DL': ['DL1', 'DL2', 'DL3', 'DL4'],
    'DL2020': [1, 2, 3, 4]
})

# =============================================================================
# SECTION 3: Cartesian Product of 2019 and 2020 DLs
# =============================================================================

# Add dummy keys for cross join
a_table['_key'] = 1
b_table['_key'] = 1

# Perform Cartesian product using merge on dummy key
cartesian_dl = pd.merge(a_table, b_table, on='_key', suffixes=('_2019', '_2020')).drop(columns=['_key', 'DL_2020'])

# Reorder columns to match intent
cartesian_dl = cartesian_dl[['DL_2019', 'DL2019', 'DL', 'DL2020']]
cartesian_dl.columns = ['DL_2019', 'DL2019', 'DL_2020', 'DL2020']

# Display the result inline
print("▶ Cartesian Product of DL2019 and DL2020:")
display(cartesian_dl)

# =============================================================================
# SECTION 4: Visualization (Heatmap of Transition Matrix)
# =============================================================================

# Create pivot table for heatmap
pivot = cartesian_dl.pivot_table(index='DL2019', columns='DL2020', aggfunc='size', fill_value=0)

plt.figure(figsize=(8, 6))
sns.heatmap(pivot, annot=True, fmt="d", cmap="YlGnBu", cbar=False)
plt.title("Transition Matrix: DL2019 to DL2020")
plt.xlabel("DL2020")
plt.ylabel("DL2019")
plt.tight_layout()
plt.show()

# =============================================================================
# SECTION 5: Export to Excel with Data and Visualization
# =============================================================================

print("▶ Exporting data and chart to Excel...")

today_str = datetime.today().strftime("%Y%m%d")
excel_filename = f"{today_str}_DL_Cartesian_Product.xlsx"
excel_path = os.path.join(OUTPUT_DIR, excel_filename)

with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    cartesian_dl.to_excel(writer, sheet_name='Cartesian_DL', index=False)
    pivot.to_excel(writer, sheet_name='Pivot_DL')

    # Insert chart below the pivot
    workbook  = writer.book
    worksheet = writer.sheets['Pivot_DL']

    chart = workbook.add_chart({'type': 'heatmap'})

    # Since xlsxwriter doesn't support heatmaps directly, just mention
    # (Manually inserting a screenshot or plot is better in this case)
    worksheet.write('G2', '▶ Heatmap was shown in Colab (not embedded in Excel).')

print(f"Excel file saved to: {excel_path}")
